In [ ]:
import pandas as pd
import re

In [56]:
df = pd.read_csv("../Data/clean_data.csv")
df

,No,Price,Area,Bedrooms,WCs,Address,Postdate,Description
0,1,3.899000e+09,150.0,2.0,1.0,Thành phố Hồ Chí Minh,2023-12-10,"Thông sô cưc đep 5 x 30m 150m2 | Nhà cấp 4, 2..."
1,2,5.500000e+00,51.0,NaN,NaN,ÂU CƠ PHƯỜNG 9 TÂN BÌNH,2023-12-10,DT 51m2 cực to cứng nhất khu | Kết cấu 3 Tầng...
2,3,4.600000e+00,45.0,2.0,2.0,NaN,2023-12-10,"Đường đi sạch sẽ, an ninh, yên tĩnh. Khu vực ..."
3,4,7.350000e+00,41.0,NaN,NaN,LÊ HỒNG PHONG PHƯỜNG 2 QUẬN 5,2023-12-10,"Diện tích công nhận 41m, thực tế 66m ngang 3...."
4,5,NaN,96.0,2.0,1.0,NaN,2023-12-07,"sổ vuông vức, thuận tiện xây mới, phù hợp xay..."
...,...,...,...,...,...,...,...,...
1670,1671,4.900000e+00,51.0,4.0,3.0,"Phạm Tử Nghi, Lê Chân, Hải Phòng",2023-10-20,Liên hệ Mrs Tâm 0796336866 | Thông tin chi t...
1671,1672,7.500000e+00,97.0,3.0,2.0,"Trực Cát, Lê Chân, Hải Phòng",2023-10-20,Liên hệ Sàn Bán Nhà Hải Phòng 0796336866 | T...
1672,1673,4.900000e+00,33.0,NaN,NaN,Đường Số 9,2023-10-20,"Vị trí Nhà mặt tiền đang kinh doanh, gần trườ..."
1673,1674,4.200000e+00,52.0,2.0,2.0,"Lê Văn Việt, Tăng Nhơn Phú A",2023-10-20,"Thông ra Lê Văn Việt , chợ Tăng Nhơn Phú A, Lã..."


In [57]:
def extract_property_info(text):
    # Biến chứa các thông tin cần trích xuất
    info = {
        'price': None,
        'address': None,
        'area': None,
        'bedrooms': None,
        'toilets': None,
        'floors': None,
        'other_features': None
    }
    
    if isinstance(text, str):
        # Trích xuất giá
        price_match = re.search(r'Giá ([\d\.]+) tỷ', text)
        if price_match:
            info['price'] = float(price_match.group(1))
        
        # Trích xuất địa chỉ
        address_match = re.search(r'Vị trí (.+?) \|', text)
        if address_match:
            info['address'] = address_match.group(1).strip()
        
        # Trích xuất diện tích
        area_match = re.search(r'Diện tích (\d+\.?\d*)m2', text)
        if area_match:
            info['area'] = float(area_match.group(1))
        
        # Trích xuất số phòng ngủ
        bedrooms_match = re.search(r'(\d+) phòng ngủ', text)
        if bedrooms_match:
            info['bedrooms'] = int(bedrooms_match.group(1))
        
        # Trích xuất số toilet
        toilets_match = re.search(r'(\d+) toilet', text)
        if toilets_match:
            info['toilets'] = int(toilets_match.group(1))
        
        # Trích xuất số tầng
        floors_match = re.search(r'(\d+) tầng', text)
        if floors_match:
            info['floors'] = int(floors_match.group(1))
        
        # Trích xuất các đặc điểm khác
        other_features_match = re.search(r'(\d+ tầng, .+?) \|', text)
        if other_features_match:
            info['other_features'] = other_features_match.group(1).strip()
    
    return info

# Đọc dữ liệu từ file CSV
df = pd.read_csv('../Data/clean_data.csv')

# Tạo các cột mới từ thông tin trích xuất
df['price'] = df['Description'].apply(lambda x: extract_property_info(x)['price'] if pd.notnull(x) else None)
df['address'] = df['Description'].apply(lambda x: extract_property_info(x)['address'] if pd.notnull(x) else None)
df['area'] = df['Description'].apply(lambda x: extract_property_info(x)['area'] if pd.notnull(x) else None)
df['bedrooms'] = df['Description'].apply(lambda x: extract_property_info(x)['bedrooms'] if pd.notnull(x) else None)
df['toilets'] = df['Description'].apply(lambda x: extract_property_info(x)['toilets'] if pd.notnull(x) else None)
df['floors'] = df['Description'].apply(lambda x: extract_property_info(x)['floors'] if pd.notnull(x) else None)
df['other_features'] = df['Description'].apply(lambda x: extract_property_info(x)['other_features'] if pd.notnull(x) else None)

# Chỉ lưu các cột đã trích xuất vào file CSV mới
extracted_columns = df[['price', 'address', 'area', 'bedrooms', 'toilets', 'floors', 'other_features']]
extracted_columns.to_csv('../Data/extracted_data_description.csv', index=False)

print("Extraction complete. Results saved to '../Data/extracted_data_description.csv'.")


Extraction complete. Results saved to '../Data/extracted_data_description.csv'.
